## 데이터 선별

 - 확보한 의약품 데이터에 존재하는 불필요한 컬럼이나 중복을 제거합니다.  
 - 필요한 경우 데이터 내용을 변경

### > 라이브러리 import 및 환경설정

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import display
from tqdm.notebook import tqdm, trange
from pathlib import Path
import pandas as pd
import json
import os

### > csv 파일 불러오기

In [ ]:

raw_dfs = {
    "의약품허가정보": {
        "허가목록": None,
        "허가상세정보": None,
        # "주성분상세정보": None,   #제외사유: 존재하는 컬럼이 모두 다른 API에 존재하거나 불필요함
    },
    "낱알식별정보": {
        "낱알식별정보": None,
    },
    "DUR성분정보": {
        # "병용금기": None,         #제외사유: 품목정보-병용금기의 병용금기의약품에대한 복합제 정보가 주어져있지 않거나(누락), 단일 의약품에 대해 복합 의약품으로 표시되어 있는 등의 문제점이 존재함
        "특정연령대금기": None,
        "임부금기": None,
        "용량주의": None,
        "투여기간주의": None,
        # "효능군중복": None,       #제외사유: 존재하는 컬럼이 모두 다른 API에 존재하거나 불필요함
        # "노인주의": None,         #제외사유: 존재하는 컬럼이 모두 다른 API에 존재하거나 불필요함
    },
    "DUR품목정보": {
        "병용금기": None,
        "특정연령대금기": None,
        "임부금기": None,
        "용량주의": None,
        "투여기간주의": None,
        "효능군중복": None,
        "서방정분할주의": None,
        "노인주의": None,
        # "DUR품목정보": None,      #제외사유: 존재하는 컬럼이 모두 다른 API에 존재하거나 불필요함
    },
}

In [ ]:
# /dat 디렉토리 내, 이름 형식이 yyyy-mm-dd_xxhxxmxxs 인 디렉토리 하나를 선택해야 함
import tkinter as tk
from tkinter import filedialog
window = tk.Tk()
window.file = filedialog.askdirectory()

# 진행도 표시 위젯
pbar = tqdm([op for service in raw_dfs for op in raw_dfs[service]], desc= "전체 진행도")

for service, operations in raw_dfs.items():
    for operation, dat in operations.items():
        filepath = f"{window.file}\\csv\\raw\\{service}-{operation}.csv"
        operations[operation] = pd.read_csv(filepath, keep_default_na=False)

        pbar.update(1)

window.destroy()

### > 불필요, 중복 컬럼 제거

#### >> 의약품허가정보-허가목록

    ***제거대상컬럼***
    품목명(국문)        ITEM_NAME
    품목명(영문)        ITEM_ENG_NAME
    품목일련번호        PRDLST_STDR_CODE    (품목기준번호 아님)
    품목허가번호        PRDUCT_PRMISN_NO
    품목허가일자        ITEM_PERMIT_DATE
    전문일반구분        SPCLTY_PBLC
    주성분수            ITEM_INGR_CNT
    주성분명            ITEM_INGR_NAME
    EDI코드(보험코드)   EDI_CODE
    허가/신고구분       PERMIT_KIND_CODE
    취하/취소상태       CANCEL_NAME
    취하/취소일자       CANCEL_DATE
    업종                INDUTY
    업체명(국문)        ENTP_NAME
    업체명(영문)        ENTP_ENG_NAME
    업체코드            ENTP_SEQ
    업체허가번호        ENTP_NO
    사업자등록번호      BIZRNO

In [ ]:
# 목표 데이터프레임 레퍼런스
target = raw_dfs["의약품허가정보"]["허가목록"]

# 제거할 컬럼
target_cols = [
    "ITEM_NAME", "ITEM_ENG_NAME", "PRDLST_STDR_CODE", "PRDUCT_PRMISN_NO",
    "ITEM_PERMIT_DATE", "SPCLTY_PBLC", "ITEM_INGR_CNT", "ITEM_INGR_NAME",
    "EDI_CODE", "PERMIT_KIND_CODE", "CANCEL_NAME", "CANCEL_DATE", "INDUTY",
    "ENTP_NAME", "ENTP_ENG_NAME", "ENTP_SEQ", "ENTP_NO", "BIZRNO"
]

target.drop(target_cols, axis=1, inplace=True)
target.info()
display(target)

#### >> 의약품허가정보-허가상세정보

    ***제거대상컬럼***
    효능효과(링크)              EE_DOC_ID
    용법용량(링크)              UD_DOC_ID
    주의사항(링크)              NB_DOC_ID
    신약                        NEWDRUG_CLASS_NAME
    바코드                      BAR_CODE
    재심사대상여부              REEXAM_TARGET
    재심사기간                  REEXAM_DATE
    제조방법                    DOC_TEXT
    완제/원료구분               MAKE_MATERIAL_FLAG
    총량                        TOTAL_CONTENT
    주의사항(전문) 문서데이터   PN_DOC_DATA
    업종                        INDUTY_TYPE
    업체허가번호                ENTP_NO
    변경일자                    CHANGE_DATE
    변경이력                    GBN_NAME
    사업자등록번호              BIZRNO

In [ ]:
# 목표 데이터프레임 레퍼런스
target = raw_dfs["의약품허가정보"]["허가상세정보"]

# 제거할 컬럼
target_cols = [
    "EE_DOC_ID", "UD_DOC_ID", "NB_DOC_ID", "NEWDRUG_CLASS_NAME", 
    "REEXAM_TARGET", "REEXAM_DATE", "DOC_TEXT", "MAKE_MATERIAL_FLAG",
    "TOTAL_CONTENT", "PN_DOC_DATA", "INDUTY_TYPE", "ENTP_NO",
    "CHANGE_DATE", "GBN_NAME", "BIZRNO", "BAR_CODE"
]

target.drop(target_cols, axis=1, inplace=True)
target.info()
display(target)

#### >> 낱알식별정보-낱알식별정보

    ***제거대상컬럼***
    품목명(국문)                ITEM_NAME
    품목명(영문)                ITEM_ENG_NAME
    품목허가일자                ITEM_PERMIT_DATE
    전문일반구분                ETC_OTC_NAME
    분류번호                    CLASS_NO
    분류명                      CLASS_NAME
    보험코드(EDI코드)           EDI_CODE
    성상                        CHART
    크기(장축)                  LENG_LONG
    크기(단축)                  LENG_SHORT
    크기(두께)                  THICK
    약학정보원 이미지 생성일     IMG_REGIST_TS
    마크내용(앞)                MARK_CODE_FRONT_ANAL
    마크내용(뒤)                MARK_CODE_BACK_ANAL
    마크이미지(앞)              MARK_CODE_FRONT_IMG
    마크이미지(뒤)              MARK_CODE_BACK_IMG
    마크코드(앞)                MARK_CODE_FRONT
    마크코드(뒤)                MARK_CODE_BACK
    변경일자                    CHANGE_DATE
    업체명(국문)                ENTP_NAME
    업체코드                    ENTP_SEQ
    사업자등록번호              BIZRNO

In [ ]:
# 목표 데이터프레임 레퍼런스
target = raw_dfs["낱알식별정보"]["낱알식별정보"]

# 제거할 컬럼
target_cols = [
    "ITEM_NAME", "ITEM_ENG_NAME", "ITEM_PERMIT_DATE", "ETC_OTC_NAME", "CLASS_NO",
    "CLASS_NAME", "EDI_CODE", "CHART", "LENG_LONG", "LENG_SHORT", "THICK",
    "IMG_REGIST_TS", "MARK_CODE_FRONT_ANAL", "MARK_CODE_BACK_ANAL",
    "MARK_CODE_FRONT_IMG", "MARK_CODE_BACK_IMG", "MARK_CODE_FRONT", "MARK_CODE_BACK",
    "CHANGE_DATE", "ENTP_NAME", "ENTP_SEQ", "BIZRNO"
]

target.drop(target_cols, axis=1, inplace=True)
target.info()
display(target)

#### >> DUR품목정보

    
    ***** 병용금기 제외 컬럼 *****
    DUR유형코드                 TYPE_CODE
    DUR유형                     TYPE_NAME
    DUR번호                     DUR_SEQ
    품목명                      ITEM_NAME
    업체명                      ENTP_NAME
    주성분                      MAIN_INGR
    성상                        CHART
    제형코드                    FORM_CODE
    변경일자                    CHANGE_DATE
    전문/일반코드               ETC_OTC_CODE
    전문/일반구분               ETC_OTC_NAME
    분류코드                    CLASS_CODE
    분류                        CLASS_NAME
    품목허가일자                ITEM_PERMIT_DATE
    병용금기약물-DUR번호        MIXTURE_DUR_SEQ
    병용금기약물-품목명         MIXTURE_ITEM_NAME
    병용금기약물-업체명         MIXTURE_ENTP_NAME
    병용금기약물-주성분         MIXTURE_MAIN_INGR
    병용금기약물-성상           MIXTURE_CHART
    병용금기약물-제형코드       MIXTURE_FORM_CODE
    병용금기약물-전문/일반코드  MIXTURE_ETC_OTC_CODE
    병용금기약물-전문/일반구분  MIXTURE_ETC_OTC_NAME
    병용금기약물-분류코드       MIXTURE_CLASS_CODE
    병용금기약물-분류           MIXTURE_CLASS_NAME
    병용금기약물-변경일자       MIXTURE_CHANGE_DATE
    병용금기약물-품목허가일자   MIXTURE_ITEM_PERMIT_DATE
    고시일자                    NOTIFICATION_DATE
    사업자등록번호              BIZRNO
    

    ***** 서방정분할주의 제외 컬럼 *****
    DUR유형                     TYPE_NAME
    품목명                      ITEM_NAME
    업체명                      ENTP_NAME
    주성분                      MAIN_INGR
    성상                        CHART
    변경일자                    CHANGE_DATE
    전문/일반구분               ETC_OTC_NAME
    분류코드                    CLASS_CODE
    분류                        CLASS_NAME
    품목허가일자                ITEM_PERMIT_DATE
    사업자등록번호              BIZRNO
    

    ***** 효능군중복 제외 컬럼 *****
    DUR유형                     TYPE_NAME
    DUR번호                     DUR_SEQ
    품목명                      ITEM_NAME
    업체명                      ENTP_NAME
    주성분                      MAIN_INGR
    전체성분명                  INGR_ENG_NAME_FULL
    성상                        CHART
    제형코드                    FORM_CODE
    제형구분                    FORM_CODE_NAME
    (제형(FORM_NAME)은 제외 x)
    변경일자                    CHANGE_DATE
    전문/일반코드               ETC_OTC_CODE
    전문/일반구분               ETC_OTC_NAME
    분류코드                    CLASS_CODE
    분류                        CLASS_NAME
    품목허가일자                ITEM_PERMIT_DATE
    고시일자                    NOTIFICATION_DATE
    사업자등록번호              BIZRNO

    *****  그 외 제외 컬럼 *****
    DUR유형                     TYPE_NAME
    품목명                      ITEM_NAME
    업체명                      ENTP_NAME
    전체성분명                  INGR_ENG_NAME_FULL
    주성분                      MAIN_INGR
    성상                        CHART
    변경일자                    CHANGE_DATE
    전문/일반구분               ETC_OTC_NAME
    분류코드                    CLASS_CODE
    분류                        CLASS_NAME
    품목허가일자                ITEM_PERMIT_DATE
    고시일자                    NOTIFICATION_DATE
    

In [ ]:
for operation, target in tqdm(raw_dfs["DUR품목정보"].items()):
    # 제거할 컬럼 결정
    if(operation == "병용금기"):
        target_cols = [
            "TYPE_CODE", "TYPE_NAME", "DUR_SEQ", "ITEM_NAME", "ENTP_NAME", "MAIN_INGR", "CHART",
            "FORM_CODE", "CHANGE_DATE", "ETC_OTC_CODE", "ETC_OTC_NAME", "CLASS_CODE",
            "CLASS_NAME", "ITEM_PERMIT_DATE", "MIXTURE_DUR_SEQ", "MIXTURE_ITEM_NAME",
            "MIXTURE_ENTP_NAME", "MIXTURE_MAIN_INGR", "MIXTURE_CHART", "MIXTURE_FORM_CODE",
            "MIXTURE_ETC_OTC_CODE", "MIXTURE_ETC_OTC_NAME", "MIXTURE_CLASS_CODE", "MIXTURE_CLASS_NAME", 
            "MIXTURE_CHANGE_DATE", "MIXTURE_ITEM_PERMIT_DATE", "NOTIFICATION_DATE", "BIZRNO"
        ]
    elif(operation == "서방정분할주의"):
        target_cols = [
            "TYPE_NAME", "ITEM_NAME", "ENTP_NAME", "MAIN_INGR", "CHART", 
            "CHANGE_DATE", "ETC_OTC_NAME", "CLASS_CODE", "CLASS_NAME",
            "ITEM_PERMIT_DATE", "BIZRNO"
        ]
    elif(operation == "효능군중복"):
        target_cols = [
            "TYPE_NAME", "DUR_SEQ", "ITEM_NAME", "ENTP_NAME", "MAIN_INGR", "INGR_ENG_NAME_FULL",
            "CHART", "FORM_CODE", "FORM_CODE_NAME", "CHANGE_DATE", "ETC_OTC_CODE", "ETC_OTC_NAME", 
            "CLASS_CODE", "CLASS_NAME", "ITEM_PERMIT_DATE",
            "NOTIFICATION_DATE", "BIZRNO"
        ]
    else:
        target_cols = [
            "TYPE_NAME", "ITEM_NAME", "ENTP_NAME", "INGR_ENG_NAME_FULL", "MAIN_INGR",
            "CHART", "CHANGE_DATE", "ETC_OTC_NAME", "CLASS_CODE", "CLASS_NAME", 
            "ITEM_PERMIT_DATE", "NOTIFICATION_DATE",
        ]

    # 목표 데이터프레임 레퍼런스
    target = raw_dfs["DUR품목정보"][operation]

    target.drop(target_cols, axis=1, inplace=True)

#### >> DUR성분정보 및 품목정보 통합

DUR성분정보 중  
~~병용금기~~  
~~노인주의~~  
~~효능군중복~~  
API 데이터는 중복 컬럼을 제외하면 유용한 컬럼이 없으므로 배제

    DUR성분정보-임부금기
    => 임부금기등급(GRADE) 추출하여 품목정보에 추가

In [ ]:
meditem = raw_dfs["DUR품목정보"]["임부금기"]
medingr = raw_dfs["DUR성분정보"]["임부금기"]
meditem["GRADE"] = pd.Series()

for idx, row in tqdm(meditem.iterrows()):
    ingr_code = row.loc["INGR_CODE"]
    grade = medingr[medingr["INGR_CODE"]==ingr_code].iloc[0]["GRADE"]
    meditem.at[idx, "GRADE"] = grade

    DUR성분정보-특정연령대금기
    => 기준연령(AGE_BASE) 추출하여 품목정보에 추가

In [ ]:
meditem = raw_dfs["DUR품목정보"]["특정연령대금기"]
medingr = raw_dfs["DUR성분정보"]["특정연령대금기"]
meditem["AGE_BASE"] = pd.Series()

for idx, row in tqdm(meditem.iterrows()):
    ingr_code = row.loc["INGR_CODE"]
    grade = medingr[medingr["INGR_CODE"]==ingr_code].iloc[0]["AGE_BASE"]
    meditem.at[idx, "AGE_BASE"] = grade

    DUR성분정보-용량주의
    => 1일최대용량(MAX_QTY) 추출하여 품목정보에 추가

In [ ]:
meditem = raw_dfs["DUR품목정보"]["용량주의"]
medingr = raw_dfs["DUR성분정보"]["용량주의"]
meditem["MAX_QTY"] = pd.Series()

for idx, row in tqdm(meditem.iterrows()):
    ingr_code = row.loc["INGR_CODE"]
    grade = medingr[medingr["INGR_CODE"]==ingr_code].iloc[0]["MAX_QTY"]
    meditem.at[idx, "MAX_QTY"] = grade

    DUR성분정보-투여기간주의
    => 최대투여기간(MAX_DOSAGE_TERM) 추출하여 품목정보에 추가

In [ ]:
meditem = raw_dfs["DUR품목정보"]["투여기간주의"]
medingr = raw_dfs["DUR성분정보"]["투여기간주의"]
meditem["MAX_DOSAGE_TERM"] = pd.Series()

for idx, row in tqdm(meditem.iterrows()):
    ingr_code = row.loc["INGR_CODE"]
    grade = medingr[medingr["INGR_CODE"]==ingr_code].iloc[0]["MAX_DOSAGE_TERM"]
    meditem.at[idx, "MAX_DOSAGE_TERM"] = grade

### > 의약품 레코드 불일치 해결

#### >> 레코드 내 완전 일치 레코드 삭제

    완전 일치 레코드란 인덱스를 제외한 모든 컬럼의 값이 동일한 레코드를 말합니다.

In [ ]:
##### 모든 행이 일치하는 레코드 삭제 #####
raw_dfs["의약품허가정보"]["허가목록"].drop_duplicates(inplace=True)
raw_dfs["의약품허가정보"]["허가상세정보"].drop_duplicates(inplace=True)
raw_dfs["낱알식별정보"]["낱알식별정보"].drop_duplicates(inplace=True)

    낱알식별정보-낱알식별정보 데이터프레임 중  
    ITEM_SEQ가 중복이지만, 모든 컬럼이 완전히 일치하지는 않는 경우가 있는지 확인

In [ ]:
df = raw_dfs["의약품허가정보"]["허가목록"].copy()

# 중복 기준: ITEM_SEQ
# 중복된 모든 행(첫 행을 제외하지 않은)을 추출
dup_df = df.loc[df.duplicated(['ITEM_SEQ']), :]
dup_item_seq_set_df = dup_df['ITEM_SEQ'].drop_duplicates()

loose = []
for item_seq in tqdm(dup_item_seq_set_df):
    loose.append(df[(df['ITEM_SEQ']==item_seq)])
if loose:
    all_dup_df = pd.concat(loose)
    print(f"중복 품목기준번호 수: {len(dup_df['ITEM_SEQ'].unique())}")
    print(f"중복 레코드 수: {len(all_dup_df)}")
    display(all_dup_df)
else:
    print("'의약품허가정보-허가목록' 에서")
    print("모든 컬럼이 완전히 동일한 레코드를 제외하면 중복된 ITEM_SEQ를 가지는 레코드가 존재하지 않습니다!")

    의약품허가정보-허가목록 데이터프레임 중  
    ITEM_SEQ가 중복이지만, 모든 컬럼이 완전히 일치하지는 않는 경우가 있는지 확인

In [ ]:
df = raw_dfs["의약품허가정보"]["허가상세정보"].copy()

# 중복 기준: ITEM_SEQ
# 중복된 모든 행(첫 행을 제외하지 않은)을 추출
dup_df = df.loc[df.duplicated(['ITEM_SEQ']), :]
dup_item_seq_set_df = dup_df['ITEM_SEQ'].drop_duplicates()

loose = []
for item_seq in tqdm(dup_item_seq_set_df):
    loose.append(df[(df['ITEM_SEQ']==item_seq)])
if loose:
    all_dup_df = pd.concat(loose)
    print(f"중복 품목기준번호 수: {len(dup_df['ITEM_SEQ'].unique())}")
    print(f"중복 레코드 수: {len(all_dup_df)}")
    display(all_dup_df)
else:
    print("'의약품허가정보-허가상세정보' 에서")
    print("모든 컬럼이 완전히 동일한 레코드를 제외하면 중복된 ITEM_SEQ를 가지는 레코드가 존재하지 않습니다!")

    의약품허가정보-허가상세정보 데이터프레임 중  
    ITEM_SEQ가 중복이지만, 모든 컬럼이 완전히 일치하지는 않는 경우가 있는지 확인

In [ ]:
df = raw_dfs["낱알식별정보"]["낱알식별정보"].copy()

# 중복 기준: ITEM_SEQ
# 중복된 모든 행(첫 행을 제외하지 않은)을 추출
dup_df = df.loc[df.duplicated(['ITEM_SEQ']), :]
dup_item_seq_set_df = dup_df['ITEM_SEQ'].drop_duplicates()

loose = []
for item_seq in tqdm(dup_item_seq_set_df):
    loose.append(df[(df['ITEM_SEQ']==item_seq)])
if loose:
    all_dup_df = pd.concat(loose)
    print(f"중복 품목기준번호 수: {len(dup_df['ITEM_SEQ'].unique())}")
    print(f"중복 레코드 수: {len(all_dup_df)}")
    display(all_dup_df)
    print("'낱알식별정보-낱알식별정보' 의 경우")
    print("중복된 ITEM_SEQ 값을 가지는 레코드들이 다른 식별 정보를 가집니다.")
else:
    print("'낱알식별정보-낱알식별정보' 에서")
    print("모든 컬럼이 완전히 동일한 레코드를 제외하면 중복된 ITEM_SEQ를 가지는 레코드가 존재하지 않습니다!")

#### >> 불일치 레코드 해결

    `의약품허가정보-허가상세정보` 를 중심으로 불일치 레코드를 제거합니다

In [ ]:
set_of_허가상세정보 = set(raw_dfs["의약품허가정보"]["허가상세정보"]["ITEM_SEQ"])
set_of_허가목록 = set(raw_dfs["의약품허가정보"]["허가목록"]["ITEM_SEQ"])
set_of_낱알식별정보 = set(raw_dfs["낱알식별정보"]["낱알식별정보"]["ITEM_SEQ"])

In [ ]:
print(f"허가상세정보 ITEM_SEQ 집합 크기         : {len(set_of_허가상세정보)}")
print(f"허가목록 ITEM_SEQ 집합 크기             : {len(set_of_허가목록)}")

inter1 = set_of_허가상세정보.intersection(set_of_허가목록)
print(f"교집합 크기                            : {len(inter1)}")

    교집합의 크기가 허가목록 집합의 크기와 같은 것으로 미루어 보아
    허가상세정보 ⊃ 허가목록입니다.
    허가목록의 컬럼이 필수적이지 않으므로, 허가목록에 존재하지 않는 허가상세정보 레코드를 삭제할 필요는 없습니다

    품목기준번호를 기준으로 허가상세정보에 허가목록의 레코드를 이어 붙여야합니다 => Outer join

In [ ]:
medicine_info = pd.merge(raw_dfs["의약품허가정보"]["허가목록"], raw_dfs["의약품허가정보"]["허가상세정보"], left_on='ITEM_SEQ', right_on='ITEM_SEQ', how='outer')
medicine_info

    허가상세정보 레코드 수  = 49,717
    허가목록 레코드 수      = 49,698
    아우터 조인 레코드 수   = 49,717

    한편, 낱알식별정보에 대해서도 레코드 불일치 여부를 확인해야합니다.

In [ ]:
print(f"낱알식별정보 ITEM_SEQ 집합 크기         : {len(set_of_낱알식별정보)}")

inter2 = set_of_허가상세정보.intersection(set_of_낱알식별정보)
print(f"허가상세정보-낱알식별정보 교집합 크기   : {len(inter2)}")

sub = set_of_낱알식별정보 - set_of_허가상세정보
print(f"차집합 크기                             : {len(sub)}")

    차집합에 해당하는 품목기준번호를 임의로 뽑아서 식약처에 검색해본 결과 결과가 나오지 않는 것으로 미루어보아.
    해당 의약품들은 식약처에서 더 이상 관리하지 의약품인 것 같습니다.
    
    따라서, 낱알식별정보로부터 해당 품목기준번호를 가지고 있는 레코드를 삭제해야합니다.

In [ ]:
phill_appearance = raw_dfs["낱알식별정보"]["낱알식별정보"].copy()
phill_appearance = phill_appearance[phill_appearance["ITEM_SEQ"].isin(inter2)]
phill_appearance

    DUR품목정보 데이터프레임 중 item_seq가 medicine_info 데이터프레임에 존재하지 않는 레코드를 삭제합니다.

In [ ]:
set_of_medicine_info = set(medicine_info["ITEM_SEQ"])

for operation, df in tqdm(raw_dfs["DUR품목정보"].items()):
    set_of_operation = set(df["ITEM_SEQ"])
    inter = set_of_medicine_info.intersection(set_of_operation)
    df = df[df["ITEM_SEQ"].isin(inter)]

    print(f"{operation}: {len(set_of_operation)-len(inter)} 개 품목 제외")

dur_elder = raw_dfs["DUR품목정보"]["노인주의"]
dur_concomitant = raw_dfs["DUR품목정보"]["병용금기"]
dur_xrsplit = raw_dfs["DUR품목정보"]["서방정분할주의"]
dur_dosage = raw_dfs["DUR품목정보"]["용량주의"]
dur_pregnancy = raw_dfs["DUR품목정보"]["임부금기"]
dur_period = raw_dfs["DUR품목정보"]["투여기간주의"]
dur_age = raw_dfs["DUR품목정보"]["특정연령대금기"]
dur_efficacyduplication = raw_dfs["DUR품목정보"]["효능군중복"]


### > medicine_info 분리

- medinfo_filter: 검색 필터로 사용되는 컬럼
- medinfo_nonfileter: 검색 필터로 사용되지 않는 컬럼

In [ ]:
#ITEM_SEQ를 제외한 필터 리스트
filter_list = ["ITEM_NAME", "ITEM_ENG_NAME", "ENTP_NAME", "ENTP_ENG_NAME", "MAIN_ITEM_INGR", "INGR_NAME", "MAIN_INGR_ENG"]

#ITEM_SEQ 컬럼은 두 데이터프레임에 모두 포함되어야 한다.
medinfo_filter = medicine_info[["ITEM_SEQ"] + filter_list].copy()
medinfo_nonfilter = medicine_info.copy().drop(columns = filter_list)

### > 데이터프레임 컬럼 형식 수정

In [ ]:
dfs = {
    "medinfo_filter": medinfo_filter, 
    "medinfo_nonfilter": medinfo_nonfilter, 
    "phill_appearance": phill_appearance, 
    "dur_elder": dur_elder,
    "dur_concomitant": dur_concomitant, 
    "dur_xrsplit": dur_xrsplit, 
    "dur_dosage": dur_dosage, 
    "dur_pregnancy": dur_pregnancy, 
    "dur_period": dur_period, 
    "dur_age": dur_age, 
    "dur_efficacyduplication": dur_efficacyduplication
}

#### >> 개행 문자 제거

 - 문서 데이터 컬럼(EE_DOC_DATA, UD_DOC_DATA, NB_DOC_DATA)를 제외한 나머지 모든 컬럼에 존재하는 개행문자(\n, \r)를 제거합니다.


In [ ]:
for name, df in tqdm(dfs.items()):
    for col in df.columns:
        if col not in ["EE_DOC_DATA", "UD_DOC_DATA", "NB_DOC_DATA"]:
            df[col].replace("\n", "", regex=True, inplace=True)
            df[col].replace("\r", "", regex=True, inplace=True)

#### >> 알약식별정보 형식 수정

앞 뒷면의 음각정보 각각에 대해
1. "분할선"을 공백 한 칸으로 대체  
2. "십자"를 삭제
3. "마크" 앞, 뒤에 공백 삽입
4. 바로 앞 또는 맨 뒤 공백 제거 / 연속 공백을 공백 한 칸으로 대체

In [ ]:
phill_appearance["PRINT_FRONT"].replace("분할선", " ", regex=True, inplace=True)
phill_appearance["PRINT_FRONT"].replace("십자", "", regex=True, inplace=True)
phill_appearance["PRINT_FRONT"].replace("마크", " 마크 ", regex=True, inplace=True)
phill_appearance["PRINT_FRONT"] = phill_appearance["PRINT_FRONT"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

phill_appearance["PRINT_BACK"].replace("분할선", " ", regex=True, inplace=True)
phill_appearance["PRINT_BACK"].replace("십자", "", regex=True, inplace=True)
phill_appearance["PRINT_BACK"].replace("마크", " 마크 ", regex=True, inplace=True)
phill_appearance["PRINT_FRONT"] = phill_appearance["PRINT_BACK"].apply(lambda x: ' '.join(x.split()) if x is isinstance(x, str) else x)

phill_appearance[["PRINT_FRONT", "PRINT_BACK"]]

#### >> 검색 필터 컬럼 내 구분자 앞뒤로 공백 추가

 - 검색 필터를 포함하는 medinfo_filter 그리고 phill_appearance 데이터프레임에 대해서 수행
 - | 와 / 앞, 뒤로 공백 문자를 삽입한 뒤 연속 공백을 제거한다.

In [ ]:
# MAIN_ITEM_INGR , INGR_NAME , MAIN_INGR_ENG
medinfo_filter["MAIN_ITEM_INGR"].replace("\|", " | ", regex=True, inplace=True)
medinfo_filter["MAIN_ITEM_INGR"] = medinfo_filter["MAIN_ITEM_INGR"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
medinfo_filter["INGR_NAME"].replace("\|", " | ", regex=True, inplace=True)
medinfo_filter["INGR_NAME"] = medinfo_filter["INGR_NAME"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
medinfo_filter["MAIN_INGR_ENG"].replace("/", " / ", regex=True, inplace=True)
medinfo_filter["MAIN_INGR_ENG"] = medinfo_filter["MAIN_INGR_ENG"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


medinfo_filter

In [ ]:
# DRUG_SHAPE , COLOR_CLASS1 , COLOR_CLASS2
phill_appearance["DRUG_SHAPE"].replace(",", " , ", regex=True, inplace=True)
phill_appearance["DRUG_SHAPE"] = phill_appearance["DRUG_SHAPE"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
phill_appearance["COLOR_CLASS1"].replace(",", " , ", regex=True, inplace=True)
phill_appearance["COLOR_CLASS1"] = phill_appearance["COLOR_CLASS1"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
phill_appearance["COLOR_CLASS2"].replace(",", " , ", regex=True, inplace=True)
phill_appearance["COLOR_CLASS2"] = phill_appearance["COLOR_CLASS2"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
phill_appearance["FORM_CODE_NAME"].replace(",", " , ", regex=True, inplace=True)
phill_appearance["FORM_CODE_NAME"] = phill_appearance["FORM_CODE_NAME"].apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

phill_appearance

#### >> 성분명 컬럼에 포함된 원료 코드([M000000] 형식) 부분 제거

[Mxxxxxx] 형식의 성분 코드를 전부 삭제합니다.  
삭제하는 이유는 해당 성분 코드를 사용할 일이 없을 뿐더러, 제거하는 것이 더 효율적인 검색이 가능하기 때문입니다.  
삭제 대상 컬럼은 아래와 같습니다.
 - medinfo_filter 중 MAIN_ITEM_INGR, INFR_NAME
 - dur_elder 중 MIX_INGR


In [ ]:
medinfo_filter["MAIN_ITEM_INGR"].replace("\[M......\]", "", regex=True, inplace=True)
medinfo_filter["INGR_NAME"].replace("\[M......\]", "", regex=True, inplace=True)

dur_elder["MIX_INGR"].replace("\[M......\]", "", regex=True, inplace=True)
dur_elder

#### >> 모든 컬럼을 큰 따옴표로 감싸기

In [ ]:
for name, df in tqdm(dfs.items()):
    for idx, row in df.iterrows():
        for col in df.columns:
            content = row.loc[col]
            # 문자열 형식이면서, 내용이 비어있거나 공백으로만 구성되지 않은 경우에만 큰 따옴표로 감쌉니다
            # 숫자 형식 코드가 한 개인 경우 정수로 인식되므로 큰 따옴표로 감싸지지 않으며,
            # 코드 여러 개가 컴마로 구분되어 있는 경우 문자열로 인식되기 때문에 큰 따옴표로 감싸진다.
            if isinstance(content, str) and content and not content.isspace():
                df.at[idx, col] = "\"" + content + "\""

#### >> 컬럼 최대 길이 파악

In [ ]:
for name, df in dfs.items():

    print(f"at {name}")
    for col in df.columns:

        maxlen = 0
        curlen = 0
        for item in df[col]:
            curlen = len(str(item))
            if maxlen < curlen:
                maxlen = curlen

        print(f"    {col}: {maxlen}")

### 내보내기

In [ ]:
# 디렉토리 생성
dir = f"{window.file}\\csv\\picked-out\\"
if not os.path.exists(dir):
    os.makedirs(dir)

# 내보내기
medinfo_filter.to_csv(dir + "medinfo_filter.csv", index=False)
medinfo_nonfilter.to_csv(dir + "medinfo_nonfilter.csv", index=False)
phill_appearance.to_csv(dir + "phill_appearance.csv", index=False)
dur_elder.to_csv(dir + "dur_elder.csv", index=False)
dur_concomitant.to_csv(dir + "dur_concomitant.csv", index=False)
dur_xrsplit.to_csv(dir + "dur_xrsplit.csv", index=False)
dur_dosage.to_csv(dir + "dur_dosage.csv", index=False)
dur_pregnancy.to_csv(dir + "dur_pregnancy.csv", index=False)
dur_period.to_csv(dir + "dur_period.csv", index=False)
dur_age.to_csv(dir + "dur_age.csv", index=False)
dur_efficacyduplication.to_csv(dir + "dur_efficacyduplication.csv", index=False)